# Notebook para tratamento dos dados

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

#style.use('seaborn')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    from sklearn import preprocessing, metrics
# !pip install -U scikit-fuzzy
import sys
import skfuzzy as fuzz
sys.setrecursionlimit(1000)

In [3]:
data_column_names = ["Date", "Time", "Epoch", "Moteid", "Temperature", "Humidity", "Light", "Voltage"]
dplot = pd.read_csv("../data.txt", sep = ' ', names = data_column_names)
s = 50.0
S49df = dplot[dplot['Moteid']==49]
S51df = dplot[dplot['Moteid']==51]
S50df = pd.read_csv('dfImputation.csv')
S50df['3'] = s

# Removendo os outliers pelo especialista para os vizinhos do sensor
outliersIL = []
outliersILValues = []
s1 = S49df
for i, row in s1.iterrows():
    if row['Temperature'] >=26.2:
        outliersIL.append(i)
        outliersILValues.append(row['Temperature'])
# Removendo outliers no Sensor
passo3= s1.copy()
passo3= passo3.drop(outliersIL)
passo3.to_csv('S49FINAL.csv')
# Removendo os outliers pelo especialista para os vizinhos do sensor
outliersIL2 = []
outliersILV2 = []
s2 = S51df
for i, row in s2.iterrows():
    if row['Temperature'] >=26.2:
        outliersIL2.append(i)
        outliersILV2.append(row['Temperature'])
print(len(outliersIL2))
# Removendo outliers no S2
passo3_3= s2.copy()
passo3_3= passo3_3.drop(outliersIL2)
passo3_3.to_csv('S51FINAL.csv')
# Removendo os outliers pelo especialista para os vizinhos do sensor
outliersIL_3 = []
outliersILV_3 = []
s3 = S50df
for i, row in s3.iterrows():
    if row['4'] >=26.2:
        outliersIL_3.append(i)
        outliersILV_3.append(row['4'])
# Removendo outliers no sensor
passo3_2= s3.copy()
passo3_2= passo3_2.drop(outliersIL_3)
passo3_2.to_csv('S50FINAL.csv')
S49df_ = pd.read_csv('S49FINAL.csv')
S50df_ = pd.read_csv('S50FINAL.csv')
S51df_ = pd.read_csv('S51FINAL.csv')
S50df_['3'] = s

# Sistema Fuzzy

In [10]:
# Modelo Fuzzy
temp = np.arange(-26.2, 26.2, 0.1)
tempF = np.arange(-26.2, 26.2, 0.1)
# Gerando as funções de pertinência
temp_lo = fuzz.trimf(temp, [-26.2, 0.0, 5.0])  
temp_md = fuzz.trimf(temp, [0.0, 5.0, 20.0])      
temp_hi = fuzz.trimf(temp, [17.20, 22.8, 26.2]) 
mfx = fuzz.trapmf(tempF, [-26.2, 5.0, 17.20, 26.2])
tempFinal_lo = fuzz.trapmf(tempF, [-26.2, 5.0, 17.20, 26.2]) 
tempFinal_md = fuzz.trimf(tempF, [0, 5.0, 20.0]) 
tempFinal_hi = fuzz.trimf(tempF, [17.20, 22.8, 26.2])
# Fuzificação
def fuzification(i, data, data1, data2): 
    if data['3'].ix[i]== s:
        temp_level_lo0 = fuzz.interp_membership(temp, temp_lo,  data1['Temperature'][i]) # ex: [52, [51, 52]]
        temp_level_md0 = fuzz.interp_membership(temp, temp_md,  data1['Temperature'][i])
        temp_level_hi0 = fuzz.interp_membership(temp, temp_hi,  data1['Temperature'][i])
        temp_level_lo1 = fuzz.interp_membership(temp, temp_lo,  data2['Temperature'][i])
        temp_level_md1 = fuzz.interp_membership(temp, temp_md,  data2['Temperature'][i])
        temp_level_hi1 = fuzz.interp_membership(temp, temp_hi,  data2['Temperature'][i])
        active_rule1_0 = np.fmin(temp_level_lo0, temp_level_lo1)
        active_rule1_1 = np.fmin(temp_level_lo0, temp_level_md1)
        active_rule1_2 = np.fmin(temp_level_lo0, temp_level_hi1)
        active_rule1_3 = np.fmin(temp_level_md0, temp_level_lo1)
        active_rule1_4 = np.fmin(temp_level_md0, temp_level_md1)
        active_rule1_5 = np.fmin(temp_level_md0, temp_level_hi1)
        active_rule1_6 = np.fmin(temp_level_hi0, temp_level_lo1)
        active_rule1_7 = np.fmin(temp_level_hi0, temp_level_md1)
        active_rule1_8 = np.fmin(temp_level_hi0, temp_level_hi1)
        active_rule1_Final_0 = np.fmax(active_rule1_0, tempFinal_lo)
        active_rule1_Final_1 = np.fmax(active_rule1_1, tempFinal_lo)
        active_rule1_Final_2 = np.fmax(active_rule1_2, tempFinal_md)
        active_rule1_Final_3 = np.fmax(active_rule1_3, tempFinal_lo)
        active_rule1_Final_4 = np.fmax(active_rule1_4, tempFinal_hi)
        active_rule1_Final_5 = np.fmax(active_rule1_5, tempFinal_hi)
        active_rule1_Final_6 = np.fmax(active_rule1_6, tempFinal_md)
        active_rule1_Final_7 = np.fmax(active_rule1_7, tempFinal_hi)
        active_rule1_Final_8 = np.fmax(active_rule1_8, tempFinal_hi)
        r1_0 = np.fmax(active_rule1_Final_8, np.fmax(active_rule1_Final_7, np.fmax(active_rule1_Final_6, active_rule1_Final_5) ) )  
        r1_1 = np.fmax(active_rule1_Final_5, np.fmax(active_rule1_Final_3, np.fmax(active_rule1_Final_2, np.fmax(active_rule1_Final_1, active_rule1_Final_0)) ) )  
        aggregated = np.fmax(r1_0,r1_1)
        return aggregated  
def defuzz(x, mfx, mode):
    mode = mode.lower()
    x = x.ravel()
    mfx = mfx.ravel()
    n = len(x)
    assert n == len(mfx), 'Length of x and fuzzy membership function must be \ identical.'
    if 'Centroid' in mode or 'bisector' in mode:
        tot_area = mfx.sum()
        assert tot_area != 0, 'Total area is zero in defuzzification!'
        if 'Centroid' in mode:
            return centroid(x, mfx)
        elif 'bisector' in mode:
            tmp = 0
            for k in range(n):
                tmp += mfx[k]
                if tmp >= tot_area / 2.:
                    return x[k]
    elif 'mom' in mode:
        return np.mean(x[mfx == mfx.max()])

    elif 'som' in mode:
        tmp = x[mfx == mfx.max()]
        return tmp[tmp == np.abs(tmp).min()][0]
    elif 'lom' in mode:
        tmp = x[mfx == mfx.max()]
        return tmp[tmp == np.abs(tmp).max()][0]
    else:
        raise ValueError('The input for `mode`, %s, was incorrect.' % (mode))
aux = 0.0
for i, row in S50df_.iterrows():
        if S50df_['4'][i] == 0.0:            
            try:
                S50df_.at[i, '4'] = defuzz(tempF,fuzification(i, S50df_,  S49df_,S51df_ ), 'lom')
                aux = defuzz(tempF,fuzification(i, S50df_, S49df_,S51df_ ), 'lom')
            except:
                S50df_.at[i, '4'] = aux            
S50df_.to_csv('finalsensor.csv', index=False)

# Final